In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils import data

import pandas as pd
import numpy as np

import os
import os.path as op

path = '../../data/kaggle-titanic'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
setNames = ['train_RescaleClean.csv','train_label.csv','test_RescaleClean.csv']
dfs = []
for sn in setNames:
    dfs.append( pd.read_csv(op.join(path,sn), index_col=0))
    print(dfs[-1].head())
    print(dfs[-1].info())
    
with open(op.join(path,"significantFeatures.txt"), "r") as file:
    features = [f.split('\n')[0] for f in file.readlines()]
print(features)

../../data/kaggle-titanic/train_label.csv
../../data/kaggle-titanic/test_RescaleClean.csv
../../data/kaggle-titanic/train_RescaleClean.csv
../../data/kaggle-titanic/test.csv
../../data/kaggle-titanic/submission.csv
../../data/kaggle-titanic/train.csv
../../data/kaggle-titanic/significantFeatures.txt
   Pclass  Sex  SibSp  Parch  Fare_group  Embarked  Age_group  Title
0     1.0  0.0  0.125    0.0         0.0       0.0        0.2    0.0
1     0.0  1.0  0.125    0.0         0.0       0.5        0.4    0.5
2     1.0  1.0  0.000    0.0         0.0       0.0        0.2    1.0
3     0.0  1.0  0.125    0.0         0.0       0.0        0.4    0.5
4     1.0  0.0  0.000    0.0         0.0       0.0        0.4    0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    float64
 1   Sex         891 non-null    float64
 2   SibSp       

In [2]:
# Download training data from open datasets.
class customTitanic(data.Dataset):
    
    def __init__(self, df, features, y):
        
        self.df_selected = df[features] #type: pandas.core.frame.DataFrame
        self.selected = np.stack([c.values for n, c in self.df_selected.items()], axis=1).astype(np.float32) #tpye: numpy.ndarray
    
        self.y = y.values.astype(np.float32)
        
    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        
        return [self.selected[idx], self.y[idx]]

In [3]:
X, y = dfs[0], dfs[1]['Survived']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
training_data = customTitanic(X_train, features,y_train)
test_data = customTitanic(X_test, features, y_test)

batch_size = 64
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
print(test_dataloader)
for Xt, yt in test_dataloader:
    print(f"Shape of X : {Xt.shape}")
    print(f"Shape of y: {yt.shape} {yt.dtype}")
    break

Shape of X : torch.Size([64, 7])
Shape of y: torch.Size([64]) torch.float32


In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 20),
            nn.ReLU(),
            nn.Linear(20, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=1, bias=True)
  )
)


In [5]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        
        loss = loss_fn(pred.squeeze(), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), (batch+1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred.squeeze(), y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 500
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.677829  [   64/  801]
loss: 0.668002  [  128/  801]
loss: 0.674518  [  192/  801]
loss: 0.674881  [  256/  801]
loss: 0.672066  [  320/  801]
loss: 0.672198  [  384/  801]
loss: 0.677583  [  448/  801]
loss: 0.671706  [  512/  801]
loss: 0.678611  [  576/  801]
loss: 0.669760  [  640/  801]
loss: 0.670751  [  704/  801]
loss: 0.680424  [  768/  801]
loss: 0.686861  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.674988 

Epoch 2
-------------------------------
loss: 0.677676  [   64/  801]
loss: 0.667690  [  128/  801]
loss: 0.674185  [  192/  801]
loss: 0.674611  [  256/  801]
loss: 0.671744  [  320/  801]
loss: 0.671981  [  384/  801]
loss: 0.677246  [  448/  801]
loss: 0.671368  [  512/  801]
loss: 0.678383  [  576/  801]
loss: 0.669387  [  640/  801]
loss: 0.670428  [  704/  801]
loss: 0.680236  [  768/  801]
loss: 0.686743  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.674690 

Epoch 3
----------------------------

loss: 0.669071  [  192/  801]
loss: 0.670463  [  256/  801]
loss: 0.666749  [  320/  801]
loss: 0.668637  [  384/  801]
loss: 0.672083  [  448/  801]
loss: 0.666209  [  512/  801]
loss: 0.674880  [  576/  801]
loss: 0.663681  [  640/  801]
loss: 0.665440  [  704/  801]
loss: 0.677380  [  768/  801]
loss: 0.684874  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.670119 

Epoch 19
-------------------------------
loss: 0.675193  [   64/  801]
loss: 0.662625  [  128/  801]
loss: 0.668764  [  192/  801]
loss: 0.670214  [  256/  801]
loss: 0.666446  [  320/  801]
loss: 0.668436  [  384/  801]
loss: 0.671774  [  448/  801]
loss: 0.665902  [  512/  801]
loss: 0.674669  [  576/  801]
loss: 0.663339  [  640/  801]
loss: 0.665139  [  704/  801]
loss: 0.677210  [  768/  801]
loss: 0.684759  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.669845 

Epoch 20
-------------------------------
loss: 0.675053  [   64/  801]
loss: 0.662341  [  128/  801]
loss: 0.668459  [  192/  801]
loss

loss: 0.666122  [  256/  801]
loss: 0.661425  [  320/  801]
loss: 0.665120  [  384/  801]
loss: 0.666738  [  448/  801]
loss: 0.660906  [  512/  801]
loss: 0.671220  [  576/  801]
loss: 0.657747  [  640/  801]
loss: 0.660174  [  704/  801]
loss: 0.674440  [  768/  801]
loss: 0.682790  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.665362 

Epoch 37
-------------------------------
loss: 0.672751  [   64/  801]
loss: 0.657703  [  128/  801]
loss: 0.663462  [  192/  801]
loss: 0.665888  [  256/  801]
loss: 0.661136  [  320/  801]
loss: 0.664929  [  384/  801]
loss: 0.666453  [  448/  801]
loss: 0.660625  [  512/  801]
loss: 0.671023  [  576/  801]
loss: 0.657430  [  640/  801]
loss: 0.659890  [  704/  801]
loss: 0.674283  [  768/  801]
loss: 0.682674  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.665108 

Epoch 38
-------------------------------
loss: 0.672619  [   64/  801]
loss: 0.657440  [  128/  801]
loss: 0.663179  [  192/  801]
loss: 0.665656  [  256/  801]
loss

loss: 0.680672  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.660919 

Epoch 55
-------------------------------
loss: 0.670443  [   64/  801]
loss: 0.653145  [  128/  801]
loss: 0.658529  [  192/  801]
loss: 0.661817  [  256/  801]
loss: 0.656051  [  320/  801]
loss: 0.661596  [  384/  801]
loss: 0.661511  [  448/  801]
loss: 0.655787  [  512/  801]
loss: 0.667597  [  576/  801]
loss: 0.651937  [  640/  801]
loss: 0.654908  [  704/  801]
loss: 0.671558  [  768/  801]
loss: 0.680552  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.660679 

Epoch 56
-------------------------------
loss: 0.670317  [   64/  801]
loss: 0.652900  [  128/  801]
loss: 0.658264  [  192/  801]
loss: 0.661597  [  256/  801]
loss: 0.655773  [  320/  801]
loss: 0.661415  [  384/  801]
loss: 0.661246  [  448/  801]
loss: 0.655529  [  512/  801]
loss: 0.667410  [  576/  801]
loss: 0.651643  [  640/  801]
loss: 0.654637  [  704/  801]
loss: 0.671411  [  768/  801]
loss: 0.680432  [  429/  801]
Test

loss: 0.668203  [   64/  801]
loss: 0.648870  [  128/  801]
loss: 0.653887  [  192/  801]
loss: 0.657932  [  256/  801]
loss: 0.651114  [  320/  801]
loss: 0.658383  [  384/  801]
loss: 0.656875  [  448/  801]
loss: 0.651287  [  512/  801]
loss: 0.664304  [  576/  801]
loss: 0.646774  [  640/  801]
loss: 0.650121  [  704/  801]
loss: 0.668980  [  768/  801]
loss: 0.678342  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.656493 

Epoch 74
-------------------------------
loss: 0.668079  [   64/  801]
loss: 0.648640  [  128/  801]
loss: 0.653636  [  192/  801]
loss: 0.657720  [  256/  801]
loss: 0.650842  [  320/  801]
loss: 0.658207  [  384/  801]
loss: 0.656625  [  448/  801]
loss: 0.651045  [  512/  801]
loss: 0.664124  [  576/  801]
loss: 0.646495  [  640/  801]
loss: 0.649860  [  704/  801]
loss: 0.668840  [  768/  801]
loss: 0.678216  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.656266 

Epoch 75
-------------------------------
loss: 0.667956  [   64/  801]
loss

...

loss: 0.552690  [  704/  801]
loss: 0.616964  [  768/  801]
loss: 0.598729  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.579516 

Epoch 413
-------------------------------
loss: 0.610755  [   64/  801]
loss: 0.575546  [  128/  801]
loss: 0.572420  [  192/  801]
loss: 0.580740  [  256/  801]
loss: 0.543658  [  320/  801]
loss: 0.588857  [  384/  801]
loss: 0.579737  [  448/  801]
loss: 0.581168  [  512/  801]
loss: 0.596843  [  576/  801]
loss: 0.560883  [  640/  801]
loss: 0.552349  [  704/  801]
loss: 0.616789  [  768/  801]
loss: 0.598391  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.579260 

Epoch 414
-------------------------------
loss: 0.610536  [   64/  801]
loss: 0.575313  [  128/  801]
loss: 0.572157  [  192/  801]
loss: 0.580476  [  256/  801]
loss: 0.543271  [  320/  801]
loss: 0.588616  [  384/  801]
loss: 0.579498  [  448/  801]
loss: 0.580961  [  512/  801]
loss: 0.596610  [  576/  801]
loss: 0.560616  [  640/  801]
loss: 0.552007  [  704/  801]
lo

loss: 0.592655  [  576/  801]
loss: 0.556056  [  640/  801]
loss: 0.546172  [  704/  801]
loss: 0.613644  [  768/  801]
loss: 0.592252  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.574626 

Epoch 432
-------------------------------
loss: 0.606586  [   64/  801]
loss: 0.571116  [  128/  801]
loss: 0.567409  [  192/  801]
loss: 0.575698  [  256/  801]
loss: 0.536268  [  320/  801]
loss: 0.584255  [  384/  801]
loss: 0.575174  [  448/  801]
loss: 0.577225  [  512/  801]
loss: 0.592422  [  576/  801]
loss: 0.555787  [  640/  801]
loss: 0.545827  [  704/  801]
loss: 0.613470  [  768/  801]
loss: 0.591909  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.574368 

Epoch 433
-------------------------------
loss: 0.606367  [   64/  801]
loss: 0.570883  [  128/  801]
loss: 0.567143  [  192/  801]
loss: 0.575430  [  256/  801]
loss: 0.535878  [  320/  801]
loss: 0.584013  [  384/  801]
loss: 0.574933  [  448/  801]
loss: 0.577017  [  512/  801]
loss: 0.592190  [  576/  801]
lo

loss: 0.602385  [   64/  801]
loss: 0.566675  [  128/  801]
loss: 0.562336  [  192/  801]
loss: 0.570599  [  256/  801]
loss: 0.528790  [  320/  801]
loss: 0.579643  [  384/  801]
loss: 0.570603  [  448/  801]
loss: 0.573290  [  512/  801]
loss: 0.588000  [  576/  801]
loss: 0.550690  [  640/  801]
loss: 0.539239  [  704/  801]
loss: 0.610192  [  768/  801]
loss: 0.585318  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.569455 

Epoch 452
-------------------------------
loss: 0.602163  [   64/  801]
loss: 0.566441  [  128/  801]
loss: 0.562068  [  192/  801]
loss: 0.570329  [  256/  801]
loss: 0.528393  [  320/  801]
loss: 0.579400  [  384/  801]
loss: 0.570363  [  448/  801]
loss: 0.573083  [  512/  801]
loss: 0.587767  [  576/  801]
loss: 0.550422  [  640/  801]
loss: 0.538891  [  704/  801]
loss: 0.610021  [  768/  801]
loss: 0.584968  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.569196 

Epoch 453
-------------------------------
loss: 0.601941  [   64/  801]
lo

loss: 0.565478  [  256/  801]
loss: 0.521234  [  320/  801]
loss: 0.575030  [  384/  801]
loss: 0.566037  [  448/  801]
loss: 0.569373  [  512/  801]
loss: 0.583575  [  576/  801]
loss: 0.545588  [  640/  801]
loss: 0.532598  [  704/  801]
loss: 0.606948  [  768/  801]
loss: 0.578629  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.564533 

Epoch 471
-------------------------------
loss: 0.597932  [   64/  801]
loss: 0.562001  [  128/  801]
loss: 0.556960  [  192/  801]
loss: 0.565208  [  256/  801]
loss: 0.520834  [  320/  801]
loss: 0.574787  [  384/  801]
loss: 0.565797  [  448/  801]
loss: 0.569167  [  512/  801]
loss: 0.583342  [  576/  801]
loss: 0.545319  [  640/  801]
loss: 0.532248  [  704/  801]
loss: 0.606778  [  768/  801]
loss: 0.578275  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.564275 

Epoch 472
-------------------------------
loss: 0.597709  [   64/  801]
loss: 0.561767  [  128/  801]
loss: 0.556691  [  192/  801]
loss: 0.564939  [  256/  801]
lo

loss: 0.565488  [  512/  801]
loss: 0.579182  [  576/  801]
loss: 0.540519  [  640/  801]
loss: 0.525977  [  704/  801]
loss: 0.603761  [  768/  801]
loss: 0.571926  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.559663 

Epoch 490
-------------------------------
loss: 0.593734  [   64/  801]
loss: 0.557613  [  128/  801]
loss: 0.551848  [  192/  801]
loss: 0.560098  [  256/  801]
loss: 0.513280  [  320/  801]
loss: 0.570198  [  384/  801]
loss: 0.561248  [  448/  801]
loss: 0.565285  [  512/  801]
loss: 0.578951  [  576/  801]
loss: 0.540253  [  640/  801]
loss: 0.525628  [  704/  801]
loss: 0.603595  [  768/  801]
loss: 0.571572  [  429/  801]
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.559408 

Epoch 491
-------------------------------
loss: 0.593513  [   64/  801]
loss: 0.557382  [  128/  801]
loss: 0.551579  [  192/  801]
loss: 0.559829  [  256/  801]
loss: 0.512882  [  320/  801]
loss: 0.569957  [  384/  801]
loss: 0.561009  [  448/  801]
loss: 0.565081  [  512/  801]
lo

In [9]:
# torch.save(model, './model.pth')